# ELI5 Dataset - Gemini Answer Generation

This notebook processes the ELI5 (Explain Like I'm 5) dataset and generates LLM answers using Google's Gemini.

## Dataset Information
- **Source**: HuggingFace dataset `rexarski/eli5_category`
- **Period**: January 2017 - June 2021
- **Content**: Human-written questions and answers from the ELI5 subreddit
- **Purpose**: Generate one LLM answer per unique question and merge with human responses

####  **<span style="color:red">IMPORTANT: <span>**

**Workflow & Integration:**
1. This notebook: Load dataset → Test API → Generate Gemini answers → Merge with human answers
2. Model: gemini-2.5-flash
3. Each unique question gets ONE LLM answer replicated across all human responses
4. **To combine with OpenAI:**
   - Run `openai_generate_dataset_clean.ipynb` first (generates human + chatgpt answers)
   - Run this notebook (generates human + gemini answers)
   - Use the merge function in Cell 10 to combine both without duplicating human answers
   - Final dataset: human + chatgpt + gemini sources

## 1. Install and Import Required Libraries

In [ ]:
# Install required packages (run once)
# !pip install pandas numpy datasets
# !pip install google-generativeai
# !pip install gdown matplotlib seaborn tqdm
# !pip install fastparquet

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import os
from datetime import datetime

# Gemini API
import google.generativeai as genai

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 2. Set Up Gemini API Key

In [ ]:
import dotenv
dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✓ Gemini API key configured successfully")
else:
    print("✗ Warning: Gemini API key not set")

## 3. Load the ELI5 Dataset

## Configuration - Set Parameters Here

In [ ]:
num_questions_to_generate = 10
delay_between_api_calls = 1
test_sample_size = 2

gemini_model = "gemini-2.5-flash"

print("Configuration:")
print(f"  Questions to generate: {num_questions_to_generate}")
print(f"  Delay between calls: {delay_between_api_calls}s")
print(f"  Test sample size: {test_sample_size}")
print(f"  Model: {gemini_model}")

In [ ]:
path = "./human_data/output/eli5_sample.csv"

df_human = pd.read_csv(path)
print(f"✓ Dataset loaded with {len(df_human)} records")

unique_questions = df_human[['q_id', 'title']].drop_duplicates().reset_index(drop=True)
print(f"✓ Found {len(unique_questions)} unique questions")
print(f"  Average answers per question: {len(df_human) / len(unique_questions):.1f}")

In [ ]:
# Display first few rows
print("\nFirst few rows of dataset:")
print("=" * 80)
df_human[['q_id', 'title', 'text']].head(3)

## 4. Define LLM Answer Generation Function

In [ ]:
def generate_gemini_answer(question, model=None, max_retries=3):
    """
    Generate an ELI5-style answer using Gemini.
    
    Args:
        question: The question to answer
        model: Gemini model to use (default: gemini-2.5-flash)
        max_retries: Number of retry attempts on failure
    
    Returns:
        Generated answer as string, or error message if failed
    """
    if model is None:
        model = gemini_model
        
    if not GEMINI_API_KEY:
        return "ERROR: Gemini API key not configured"
    
    prompt = f"""You are answering questions in the style of the ELI5 (Explain Like I'm 5) subreddit. 
Provide a clear, simple explanation that a 5-year-old could understand, but still be informative.
Keep everything as one block of text.

Question: {question}

Answer:"""
    
    for attempt in range(max_retries):
        try:
            model_instance = genai.GenerativeModel(model)
            response = model_instance.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7,
                    max_output_tokens=1000,
                )
            )
            return response.text.strip()
        
        except Exception as e:
            print(f"  Attempt {attempt + 1} failed: {str(e)[:100]}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return f"ERROR: {str(e)}"
    
    return None

## 5. Test API with Sample Questions

In [ ]:
# Test with sample questions
print(f"Testing API with {test_sample_size} sample questions...")
print("=" * 80)

test_questions = unique_questions.head(test_sample_size)

for idx, row in test_questions.iterrows():
    question = row['title']
    print(f"\n[Test {idx + 1}] {question[:70]}...")
    answer = generate_gemini_answer(question)
    
    if answer.startswith("ERROR"):
        print(f"✗ {answer}")
    else:
        print(f"✓ Generated ({len(answer)} chars): {answer[:100]}...")
    
    time.sleep(1)

print("\n" + "=" * 80)
print("API test complete!")

## 6. Generate LLM Answer for Each Unique Question

In [ ]:
# Generate one LLM answer per unique question
llm_answers_map = {}  # Map from q_id to llm_answer

questions_to_process = unique_questions.head(min(num_questions_to_generate, len(unique_questions)))
print(f"Generating {len(questions_to_process)} LLM answers...")
print("=" * 80)

with tqdm(total=len(questions_to_process), desc="Generating answers") as pbar:
    for idx, row in questions_to_process.iterrows():
        q_id = row['q_id']
        question = row['title']
        
        llm_answer = generate_gemini_answer(question)
        
        if not llm_answer.startswith("ERROR"):
            llm_answers_map[q_id] = llm_answer
            pbar.update(1)
        else:
            print(f"\n✗ Skipped {q_id}: {llm_answer[:80]}")
        
        time.sleep(delay_between_api_calls)

print(f"\n{'=' * 80}")
print(f"✓ Successfully generated {len(llm_answers_map)} / {len(questions_to_process)} LLM answers")

## 7. Merge LLM Answers with Human Answers

In [ ]:
# Create human answers dataset with source column
df_human_labeled = df_human.copy()
df_human_labeled['source'] = 'human'

# Create LLM answers dataset from the map
# Get one row per unique question from the original data
unique_rows = df_human.drop_duplicates(subset=['q_id'])[['q_id', 'title']].reset_index(drop=True)

# Filter to only questions that have LLM answers
llm_rows_list = []
for q_id, llm_answer in llm_answers_map.items():
    # Get the first row for this q_id to use as template
    template_row = df_human[df_human['q_id'] == q_id].iloc[0].copy()
    template_row['text'] = llm_answer
    template_row['source'] = 'gemini'
    llm_rows_list.append(template_row)

df_llm_labeled = pd.DataFrame(llm_rows_list).reset_index(drop=True)

# Combine human and LLM answers
df_combined = pd.concat([df_human_labeled, df_llm_labeled], ignore_index=True)

# Shuffle for better mixing
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

print("Combined dataset summary:")
print("=" * 80)
print(f"Total rows: {len(df_combined)}")
print(f"Human answers: {(df_combined['source'] == 'human').sum()}")
print(f"Gemini answers: {(df_combined['source'] == 'gemini').sum()}")
print(f"Columns: {list(df_combined.columns)}")
print(f"\nDataset shape: {df_combined.shape}")

In [ ]:
# Show sample rows
print("\nSample rows from combined dataset:")
print("=" * 80)

for source_type in ['human', 'gemini']:
    sample = df_combined[df_combined['source'] == source_type][['q_id', 'title', 'text', 'source']].head(1)
    if not sample.empty:
        row = sample.iloc[0]
        print(f"\n{source_type.upper()}:")
        print(f"  Question: {row['title'][:60]}")
        print(f"  Answer: {row['text'][:80]}...")
        print(f"  Source: {row['source']}")

## 8. Save Merged Dataset

In [ ]:
# Create output folder
output_folder = "gemini-output"
os.makedirs(output_folder, exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save to CSV
csv_filename = os.path.join(output_folder, f"eli5_combined_dataset_{timestamp}.csv")
df_combined.to_csv(csv_filename, index=False)
print(f"✓ CSV saved to: {csv_filename}")

# Save as Parquet
parquet_filename = os.path.join(output_folder, f"eli5_combined_dataset_{timestamp}.parquet")
df_combined.to_parquet(parquet_filename, index=False)
print(f"✓ Parquet saved to: {parquet_filename}")

# Save metadata
# Save metadata
metadata = {
    'timestamp': timestamp,
    'total_rows': len(df_combined),
    'unique_questions': len(unique_questions),
    # Add int() wrapper around these sums
    'human_answers': int((df_combined['source'] == 'human').sum()),
    'chatgpt_answers': int((df_combined['source'] == 'chatgpt').sum()),
    'llm_model': 'gpt-4o-mini',
    'columns': list(df_combined.columns)
}

metadata_filename = os.path.join(output_folder, f"metadata_{timestamp}.json")
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadata saved to: {metadata_filename}")

print("\n" + "=" * 80)
print("DATASET GENERATION COMPLETE!")
print("=" * 80)

## 9. Optional: Merge OpenAI and Gemini Outputs

If you've also run the OpenAI notebook, use this cell to combine outputs without duplicating human answers.

In [ ]:
# OPTIONAL: Merge OpenAI and Gemini outputs
# This prevents duplicate human answers by only combining LLM answers from both models

import glob

def merge_openai_gemini_outputs(combined_output_folder="merged-output"):
    """
    Combines OpenAI and Gemini outputs into a single dataset.
    Keeps human answers from OpenAI output, adds only Gemini LLM answers.
    """
    
    # Check if both output folders exist
    if not os.path.exists("openai-output"):
        print("✗ OpenAI output folder not found. Run OpenAI notebook first.")
        return
    
    if not os.path.exists("gemini-output"):
        print("✗ Gemini output folder not found. Run Gemini notebook first.")
        return
    
    # Get latest files from each folder
    openai_files = glob.glob("openai-output/eli5_combined_dataset_*.csv")
    gemini_files = glob.glob("gemini-output/eli5_combined_dataset_*.csv")
    
    if not openai_files or not gemini_files:
        print("✗ Could not find combined dataset CSV files")
        return
    
    openai_csv = max(openai_files, key=os.path.getctime)  # Get newest
    gemini_csv = max(gemini_files, key=os.path.getctime)
    
    print(f"Loading OpenAI dataset: {openai_csv}")
    df_openai = pd.read_csv(openai_csv)
    
    print(f"Loading Gemini dataset: {gemini_csv}")
    df_gemini = pd.read_csv(gemini_csv)
    
    # Keep only human answers from OpenAI (avoid duplicates)
    df_human = df_openai[df_openai['source'] == 'human'].copy()
    
    # Keep only LLM answers from both
    df_openai_llm = df_openai[df_openai['source'] == 'chatgpt'].copy()
    df_gemini_llm = df_gemini[df_gemini['source'] == 'gemini'].copy()
    
    # Combine: human + chatgpt + gemini
    df_merged = pd.concat([df_human, df_openai_llm, df_gemini_llm], ignore_index=True)
    df_merged = df_merged.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Create merged output folder
    os.makedirs(combined_output_folder, exist_ok=True)
    
    # Save merged dataset
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    merged_csv = os.path.join(combined_output_folder, f"eli5_openai_gemini_merged_{timestamp}.csv")
    df_merged.to_csv(merged_csv, index=False)
    
    merged_parquet = os.path.join(combined_output_folder, f"eli5_openai_gemini_merged_{timestamp}.parquet")
    df_merged.to_parquet(merged_parquet, index=False)
    
    # Save metadata
    metadata = {
        'timestamp': timestamp,
        'total_rows': int(len(df_merged)),  # Good practice to cast this too
        # Add int() wrapper around these sums
        'human_answers': int((df_merged['source'] == 'human').sum()),
        'chatgpt_answers': int((df_merged['source'] == 'chatgpt').sum()),
        'gemini_answers': int((df_merged['source'] == 'gemini').sum()),
        'source_openai': openai_csv,
        'source_gemini': gemini_csv,
        'columns': list(df_merged.columns)
    }
    
    metadata_file = os.path.join(combined_output_folder, f"metadata_{timestamp}.json")
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n{'='*80}")
    print(f"✓ Merged dataset created successfully!")
    print(f"{'='*80}")
    print(f"Total rows: {len(df_merged)}")
    print(f"  - Human answers: {(df_merged['source'] == 'human').sum()}")
    print(f"  - ChatGPT answers: {(df_merged['source'] == 'chatgpt').sum()}")
    print(f"  - Gemini answers: {(df_merged['source'] == 'gemini').sum()}")
    print(f"\nOutput folder: {combined_output_folder}")
    print(f"Files saved: {merged_csv}, {merged_parquet}, {metadata_file}")
    
    return df_merged

# Run merge if both datasets exist
# Uncomment the line below to merge OpenAI and Gemini outputs:
df_final_merged = merge_openai_gemini_outputs()